<a href="https://colab.research.google.com/github/m3yrin/nar-latent-alignment/blob/master/colab/experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sample notebook for https://github.com/m3yrin/nar-latent-alignment
Auther: @m3yrin
* Tested on Google Colaboratory.
* Check gpu is enabled.

## Preparation

In [0]:
# check gpu is enabled.
!nvidia-smi

In [0]:
# download codes
!git clone https://github.com/m3yrin/nar-latent-alignment.git
%cd nar-latent-alignment
!ls

In [0]:
!pip install -r requirements.txt

In [0]:
# download dataset
%cd datasets
!git clone https://github.com/odashi/small_parallel_enja.git
!ls small_parallel_enja
%cd ../

In [0]:
# check common config.
!cat configs/common.jsonnet

In [0]:
%%writefile configs/common.jsonnet
# if you don't use cuda, cuda_device=-1
local cuda_device = 0;

# https://arxiv.org/pdf/2004.07437.pdf
# Our models consists of 12 self-attention
# layers, with 512 hidden size, 2048 filter size, and
# 8 attention heads per layer. We use 0.1 dropout
# for regularization. 

{
    "direction" : "ja-en",
    "train_data_path": "datasets/small_parallel_enja/train",
    "validation_data_path": "datasets/small_parallel_enja/dev",

    "embedding_dim" : 64,
    "feedforward_hidden_dim" : 2048,
    "num_layers" : 12,

    "batch_size" : 128,
    "min_count" : 2,

    "num_epochs" : 40,
    "patience": 10,
    "cuda_device" : cuda_device,
}


## CTC model

In [0]:
!cat configs/ctc.jsonnet

In [0]:
# %%writefile configs/ctc.jsonnet

In [0]:
!allennlp train -f --include-package src -s tmp_ctc configs/ctc.jsonnet

In [0]:
!allennlp evaluate --output-file tmp_ctc/output_test.json --cuda-device 0 --include-package src tmp_ctc/model.tar.gz datasets/small_parallel_enja/test

In [0]:
# building a json file for prediction
!python datasets/make_json.py -I datasets/small_parallel_enja/test.ja -O datasets/test.ja.json

In [0]:
!allennlp predict --output-file tmp_ctc/output_pred.json --cuda-device 0 --include-package src --predictor small_parallel_enja_predictor  tmp_ctc/model.tar.gz datasets/test.ja.json

### Transformer Autoregressive model (for comparison)

In [0]:
!cat configs/transformer_ar.jsonnet

In [0]:
# %%writefile configs/transformer_ar.jsonnet

In [0]:
!allennlp train -f --include-package src -s tmp_ar configs/transformer_ar.jsonnet

In [0]:
!allennlp evaluate --output-file tmp_ar/output_test.json --cuda-device 0 --include-package src tmp_ar/model.tar.gz datasets/small_parallel_enja/test

In [0]:
# !python datasets/make_json.py -I datasets/small_parallel_enja/test.ja -O datasets/test.ja.json

In [0]:
!allennlp predict --output-file tmp_ar/output_pred.json --cuda-device 0 --include-package src --predictor small_parallel_enja_predictor  tmp_ar/model.tar.gz datasets/test.ja.json